In [506]:
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 30


In [507]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf   

# Creating the teams dataframe

In [508]:
TEAMNAMES=pd.read_csv('../Data/from_sqlite_Data/Team.csv')
LEAGUENAMES= pd.read_csv('../Data/from_sqlite_Data/League.csv')
df2 = pd.read_csv('../Data/from_sqlite_Data/bigfinal.csv')
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
df2.drop(['league_id'],axis=1,inplace=True)
df2.drop(['id'],axis=1,inplace=True)

In [509]:
season = df2
FTR=[]
for i in range(len(season)):
    if season['FTHG'][i]>season['FTAG'][i]:
        FTR.append('H')
    if season['FTHG'][i]<season['FTAG'][i]:
        FTR.append('A')
    if season['FTHG'][i]==season['FTAG'][i]:
        FTR.append('D')
    
season['FTR']=FTR   

In [510]:
season=season.sort('Date')

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [511]:
season.reset_index(inplace=True)


In [512]:
season.drop(['index'],axis=1,inplace=True)

In [513]:
#season['season']=['2016/2017']*len(season)

In [514]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

In [344]:
def point_in_round_k(ALL_TEAM,i,team_index,where):
    
    home=where
    away= not home
    
    if home:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 3
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 0
    #   
    if away:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 0
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 3

In [345]:
def zero_or_big(a,b):
    if a-b>0:
        return a-b
    return 0

In [346]:
def poin_in_last_n_mathces(ALL_TEAM,i,team_index,where,round_index,n):
    if round_index<=n:
        return point_in_round_k(ALL_TEAM,i,team_index,where)
        

## Allteamsresults is a list of dataframes and each dataframe represents a specific team. To access info on a team use team index. The position of the team in the list "teams" gives  the index of the team in the "allteamsresults" list of dataframes.

## Now, lets calculate some other features

## 1) Goal scored and received - home and away.
### We will shift of one match back each calculation, which means, each row will have the present match and the statistics goals scored up to that day (before the match).

In [379]:
def calculate_for_team_i(ALL_TEAM,team_lists,team_index):
    
    #create the lists for goals 
    Goal_scored_home=[]
    Goal_received_home=[]
    Goal_scored_away=[]
    Goal_received_away=[]
    
    #total_goal
    total_goal_scored=[]
    total_goal_received=[]
    
    #create the list for averages goals
    Av_Goal_scored_home=[]
    Av_Goal_received_home=[]
   

    Av_Goal_scored_away=[]
    Av_Goal_received_away=[]
       
    #create the list for averages total goals
    Av_total_goal_scored=[]
    Av_total_goal_received=[]
    
    #create the lists for points
    Point_home=[]
    Point_away=[]

    total_point=[]

    #create the list for average Points
    Av_Point_home=[]
    Av_Point_away=[]

    Av_total_point=[]
    
    
    #Point in the last mathces
    
    T_L3M_Pnt=[]
    T_L5M_Pnt=[]
    
    H_L3M_Pnt=[]
    A_L3M_Pnt=[]
    
    H_L5M_Pnt=[]
    A_L5M_Pnt=[]
    
    
    
    #ROUND_INDEX
    round_index=[]

    
    k=1
    
    Goal_scored_home.append(0)
    Goal_received_home.append(0)
    Goal_scored_away.append(0)
    Goal_received_away.append(0)
    total_goal_scored.append(0)
    total_goal_received.append(0)
    Av_Goal_scored_home.append(0)
    Av_Goal_received_home.append(0)
    Av_Goal_scored_away.append(0)
    Av_Goal_received_away.append(0)
    Av_total_goal_scored.append(0)
    Av_total_goal_received.append(0)
    Point_home.append(0)
    Point_away.append(0)
    total_point.append(0)
    Av_Point_home.append(0)
    Av_Point_away.append(0)
    Av_total_point.append(0) 
    T_L3M_Pnt.append(0)
    T_L5M_Pnt.append(0)
    #H_L3M_Pnt.append(0)
    #A_L3M_Pnt.append(0)
    #H_L5M_Pnt.append(0)
    #A_L5M_Pnt.append(0)
    
    
    for h in allteamsresults[team_index].df['season'].unique():
    
        seasonal_index=allteamsresults[team_index].df[allteamsresults[team_index].df['season']==h].index
        Home=0
        Away=0
        total=0
        round_in=1
        tmp3=[]
        tmp5=[]
        
        for i in seasonal_index:
            
            round_index.append(round_in)
            round_index_2=round_in-1
            
            home_1=ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]
            away_1= not home_1
            

            if (round_in<=3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3)
                        tmp3.append(3)
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1) 
                        tmp3.append(1)                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0)                        
                        tmp3.append(0)
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0)
                        tmp3.append(0)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1)
                        tmp3.append(1)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3)
                        tmp3.append(3)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            #                        
            if (round_in<=5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3)
                        tmp5.append(3)
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        tmp5.append(1)
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0)
                        tmp5.append(0)
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0)
                        tmp5.append(0)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        tmp5.append(1)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3)
                        tmp5.append(3)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
                        
            '''if (round_in==3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                #
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1+T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            
            if (round_in==5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
'''
            if (round_in>3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp3.append(3)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        tmp3.append(1)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                    
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp3.append(0)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp3.append(0)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        tmp3.append(1)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp3.append(3)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                        
            if (round_in>5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp5.append(3)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        tmp5.append(1)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp5.append(0)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp5.append(0)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        tmp5.append(1)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp5.append(3)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
            
            round_in=round_in+1
            

            #HOME
            if ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_home.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_scored_home[k-1])
                Goal_received_home.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_received_home[k-1])  
                Goal_scored_away.append(Goal_scored_away[k-1])
                Goal_received_away.append(Goal_received_away[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTHG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTAG'][i])

                #Goal average
                Home=Home+1
                total=Home+Away
                Av_Goal_scored_home.append(float(Goal_scored_home[k])/Home)
                Av_Goal_received_home.append(float(Goal_received_home[k])/Home)
                Av_Goal_scored_away.append(Av_Goal_scored_away[k-1])
                Av_Goal_received_away.append(Av_Goal_received_away[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_home.append(3+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_home.append(1+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_home.append(0+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)   
                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                Point_away.append(Point_away[k-1])
                Av_Point_away.append(Av_Point_away[k-1])

            #AWAY
            if ALL_TEAM[team_index].df['HomeTeam'][i]!=team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_away.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_scored_away[k-1])
                Goal_received_away.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_received_away[k-1])
                Goal_scored_home.append(Goal_scored_home[k-1])
                Goal_received_home.append(Goal_received_home[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTAG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTHG'][i])

                #Goal average
                Away=Away+1
                total=Home+Away

                Av_Goal_scored_away.append(float(Goal_scored_away[k])/Away)
                Av_Goal_received_away.append(float(Goal_received_away[k])/Away)
                Av_Goal_scored_home.append(Av_Goal_scored_home[k-1])
                Av_Goal_received_home.append(Av_Goal_received_home[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_away.append(3+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_away.append(1+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_away.append(0+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)


                Point_home.append(Point_home[k-1])
                Av_Point_home.append(Av_Point_home[k-1])
                
                


            k=k+1

        
        Goal_scored_home[-1]=0
        Goal_received_home[-1]=0
        Goal_scored_away[-1]=0
        Goal_received_away[-1]=0
        total_goal_scored[-1]=0
        total_goal_received[-1]=0


        Av_Goal_scored_home[-1]=0
        Av_Goal_received_home[-1]=0
        Av_Goal_scored_away[-1]=0
        Av_Goal_received_away[-1]=0
        Av_total_goal_scored[-1]=0
        Av_total_goal_received[-1]=0


        Point_home[-1]=0
        Point_away[-1]=0
        Av_Point_home[-1]=0
        Av_Point_away[-1]=0

        total_point[-1]=0
        Av_total_point[-1]=0
        
        
        T_L3M_Pnt[-1]=0
        T_L5M_Pnt[-1]=0
        

    del Goal_scored_home[-1]
    del Goal_received_home[-1]
    del Goal_scored_away[-1]
    del Goal_received_away[-1]
    del total_goal_scored[-1]
    del total_goal_received[-1]


    del Av_Goal_scored_home[-1]
    del Av_Goal_received_home[-1]
    del Av_Goal_scored_away[-1]
    del Av_Goal_received_away[-1]
    del Av_total_goal_scored[-1]
    del Av_total_goal_received[-1]


    del Point_home[-1]
    del Point_away[-1]
    del Av_Point_home[-1]
    del Av_Point_away[-1]

    del total_point[-1]
    del Av_total_point[-1]
    
    del T_L3M_Pnt[-1]
    del T_L5M_Pnt[-1]
    
    ALL_TEAM[team_index].df['Round']=round_index
    
    ALL_TEAM[team_index].df['T_L3M']=T_L3M_Pnt
    ALL_TEAM[team_index].df['T_L5M']=T_L5M_Pnt
    
    ALL_TEAM[team_index].df['GS_H']=Goal_scored_home
    ALL_TEAM[team_index].df['GS_A']=Goal_scored_away
    ALL_TEAM[team_index].df['GR_H']=Goal_received_home
    ALL_TEAM[team_index].df['GR_A']=Goal_received_away
    ALL_TEAM[team_index].df['TGS']=total_goal_scored
    ALL_TEAM[team_index].df['TGR']=total_goal_received
    
    
    ALL_TEAM[team_index].df['AGS_H']=Av_Goal_scored_home
    ALL_TEAM[team_index].df['AGS_A']=Av_Goal_scored_away
    ALL_TEAM[team_index].df['AGR_H']=Av_Goal_received_home
    ALL_TEAM[team_index].df['AGR_A']=Av_Goal_received_away
    ALL_TEAM[team_index].df['ATGS']=Av_total_goal_scored
    ALL_TEAM[team_index].df['ATGR']=Av_total_goal_received

    
    ALL_TEAM[team_index].df['Pnt_H']=Point_home
    ALL_TEAM[team_index].df['Pnt_A']=Point_away
    ALL_TEAM[team_index].df['TOTPT']=total_point

    
    ALL_TEAM[team_index].df['Av_Pnt_H']=Av_Point_home
    ALL_TEAM[team_index].df['Av_pnt_A']=Av_Point_away
    ALL_TEAM[team_index].df['Av_tot_pnt']=Av_total_point
    
    
    return ALL_TEAM[team_index].df


In [348]:
#calculate_for_team_i(allteamsresults,teams,40)

In [349]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:475: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [355]:
allteamsresults[0].df.iloc[:,37:]

,FTR,Round,T_L3M,T_L5M,GS_H,GS_A,GR_H,GR_A,TGS,TGR,AGS_H,AGS_A,AGR_H,AGR_A,ATGS,ATGR,Pnt_H,Pnt_A,TOTPT,Av_Pnt_H,Av_pnt_A,Av_tot_pnt
0,A,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000
7,H,2,0,0,1,0,2,0,1,2,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,0,0,0,0.000000,0.000000,0.000000
12,D,3,0,0,1,1,2,2,2,4,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,0,0,0,0.000000,0.000000,0.000000
17,H,4,1,1,1,1,2,2,2,4,0.500000,1.000000,1.000000,2.000000,0.666667,1.333333,1,0,1,0.500000,0.000000,0.333333
46,H,5,1,0,1,2,2,4,3,6,0.500000,1.000000,1.000000,2.000000,0.750000,1.500000,1,0,1,0.500000,0.000000,0.250000
92,A,6,4,3,3,2,3,4,5,7,1.000000,1.000000,1.000000,2.000000,1.000000,1.400000,4,0,4,1.333333,0.000000,0.800000
161,D,7,6,7,3,5,3,4,8,7,1.000000,1.666667,1.000000,1.333333,1.333333,1.166667,4,3,7,1.333333,1.000000,1.166667
224,A,8,7,8,6,5,6,4,11,10,1.500000,1.666667,1.500000,1.333333,1.571429,1.428571,5,3,8,1.250000,1.000000,1.142857
315,A,9,4,7,7,5,9,4,12,13,1.400000,1.666667,1.800000,1.333333,1.500000,1.625000,5,3,8,1.000000,1.000000,1.000000
475,A,10,4,10,7,7,9,5,14,14,1.400000,1.750000,1.800000,1.250000,1.555556,1.555556,5,6,11,1.000000,1.500000,1.222222


In [354]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_TP3',
                          'H_TP5',
                            
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',
    
                          'A_TP3',
                          'A_TP5',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [ ]:
allteamsresults.[0]df.iloc[37:]

In [356]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[38:]) + list(allteamsresults[away_index].df.loc[i].iloc[38:])
    tmp.loc[i]=tobeappend

In [357]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)

In [358]:
final['DIFF_TP']=final['H_T_pnt']-final['Aw_T_pnt']
final['DIFF_HP-AP']=final['H_Pnt_H']-final['Aw_Pnt_A']

final['SUMM_HGSH+HGSH+AGRA']=final['H_GS_H']+final['Aw_GR_A']
final['SUMM_HGRH+AGSA']=final['H_GR_H']+final['Aw_GS_A']
final['SUMM_HTGS+ATGR']=final['H_TGS']+final['Aw_TGR']
final['SUMM_HTGR+ATGS']=final['H_TGR']+final['Aw_TGS']

final['DIFF_L3MP']=final['H_TP3']-final['A_TP3']
final['DIFF_L5MP']=final['H_TP5']-final['A_TP5']


final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']

final['Goal/No_Goal'] = ((final['FTHG']>0 ) * 1)*((final['FTAG']>0) * 1)
final['Over2.5/Under2.5'] = ((final['FTHG'] + final['FTAG']) >2)*1
final['Over1.5/Under1.5'] = ((final['FTHG'] + final['FTAG']) >1)*1


In [381]:
final.to_csv('../Data/from_sqlite_Data/Realfinal.csv')

In [515]:
final=pd.read_csv('../Data/from_sqlite_Data/Realfinal.csv')

# ELO

In [360]:
import io
import requests

In [423]:
k=0
for i in final.Date.unique():
    try:
        a=i.replace('00:00:00','')
        a=a.replace(' ','')
        url='http://api.clubelo.com/'+a
        s=requests.get(url).content
        c=pd.read_csv(io.StringIO(s.decode('utf-8')))
        saveto='../Data/elo/'+a
        c.to_csv(saveto)
    except:
        'EmptyDataError: No columns to parse from file'
        k=k+1
    


In [518]:
concat=[]
k=0
for i in final.Date.unique():    
    a=i.replace('00:00:00','')
    a=a.replace(' ','')
    concat.append(pd.read_csv('../Data/elo/'+a))




In [519]:
all_elo=pd.concat(concat)

In [432]:
all_elo.to_csv('../Data/elo/elo_totale')

# Merging dataframe with elo

In [520]:
import difflib

In [533]:
k=0
a=[]
b=[]
c=[]
r=list(final.HomeTeam.unique())
t=list(all_elo.Club.unique())

for i in r:
    if i in t:
        t.remove(i)
        r.remove(i)

In [536]:
for i in r:
    if i not in t:
        if len(difflib.get_close_matches(i,t))>1:
            a.append(list(difflib.get_close_matches(i,t)))
            b.append(i)
        if len(difflib.get_close_matches(i,t))==0:
            c.append(i)

In [499]:
len(a)
len(b)

45

In [537]:
for i in range(len(a)):
    print a[i]
    print b[i]
    
    
    stokecity--> Stoke
    Sporting Lokeren--->Lokeren
    Estrela Amadora--> Amadora
    Chievo Verona--->Chievo
    Sevilla FC--->

['Aarau', 'Farul']
FC Aarau
['Luzern', 'Lautern']
FC Luzern
['Bellinzona', 'Avellino']
AC Bellinzona
['Basel', 'FC Tiraspol']
FC Basel
['Vaduz', 'Cadiz']
FC Vaduz
['Valenciennes', 'Valencia']
Valenciennes FC
['Wolfsburg', 'Wolfsberg']
VfL Wolfsburg
['St Mirren', 'St Truiden']
St. Mirren
['West Ham', 'Man United']
West Ham United
['Man United', 'Sheffield United']
Manchester United
['Frankfurt', 'FSV Frankfurt']
Eintracht Frankfurt
['Lorient', 'Forest']
FC Lorient
['Cork City', 'Stoke', 'Bristol City']
Stoke City
['Newcastle', 'Man United']
Newcastle United
['Sporting', 'Sporting Fingal']
Sporting CP
['Charleroi', 'Sporting', 'Sporting Fingal']
Sporting Charleroi
['Sporting', 'Sporting Fingal', 'Lokeren']
Sporting Lokeren
['Man City', 'Bangor City']
Manchester City
['Aarau', 'Estrela Amadora', 'Sampdoria']
Amadora
['Benfica', 'Senica']
SL Benfica
['Valencia', 'Valenciennes', 'Valerenga']
Valencia CF
['Volendam', 'FC Kobenhavn']
FC Volendam
['Verona', 'Chievo']
Chievo Verona
['Osasuna', 

In [538]:
c

['Neuch\xc3\xa2tel Xamax',
 'FC Z\xc3\xbcrich',
 'Lech Pozna\xc5\x84',
 'AS Nancy-Lorraine',
 'Heart of Midlothian',
 'FC Sochaux-Montb\xc3\xa9liard',
 'Stade Rennais FC',
 'Wis\xc5\x82a Krak\xc3\xb3w',
 '\xc5\x9al\xc4\x85sk Wroc\xc5\x82aw',
 'Girondins de Bordeaux',
 'Arka Gdynia',
 'Olympique Lyonnais',
 'Polonia Bytom',
 'Hamilton Academical FC',
 'FC Bayern Munich',
 'Paris Saint-Germain',
 'FC Energie Cottbus',
 'Inverness Caledonian Thistle',
 'GKS Be\xc5\x82chat\xc3\xb3w',
 'Ruch Chorz\xc3\xb3w',
 'Borussia M\xc3\xb6nchengladbach',
 'Olympique de Marseille',
 'Vit\xc3\xb3ria Guimar\xc3\xa3es',
 'SV Werder Bremen',
 'Hertha BSC Berlin',
 'Royal Excel Mouscron',
 'RAEC Mons',
 'Naval 1\xc2\xb0 de Maio',
 '1. FC K\xc3\xb6ln',
 'Roda JC Kerkrade',
 'N.E.C.',
 'AZ',
 'RC Deportivo de La Coru\xc3\xb1a',
 'Acad\xc3\xa9mica de Coimbra',
 'Vit\xc3\xb3ria Set\xc3\xbabal',
 'Atl\xc3\xa9tico Madrid',
 'Athletic Club de Bilbao',
 'Real Betis Balompi\xc3\xa9',
 'Real Sporting de Gij\xc3\xb3n'

In [433]:
all_elo['From']=pd.to_datetime(all_elo['From'])
all_elo['To']=pd.to_datetime(all_elo['To'])

In [ ]:
lis_home=[]
for i in final.index:
    tmp=all_elo[all_elo.Club==final['HomeTeam'][i]]
    lis_home.append(tmp[tmp['To'] >= pd.to_datetime(final.Date[i])].iloc[0]['Elo'])
    

In [ ]:
lis_away=[]
for i in premier_2017.index:
    tmp=a[a.Club==premier_2017['AwayTeam'][i]]
    lis_away.append(tmp[tmp['To'] >= pd.to_datetime(premier_2017.Date[i])].iloc[0]['Elo'])
    
    

In [67]:
final.to_csv('../Data/from_sqlite_Data/Realfinal.csv')

In [68]:
final=pd.read_csv('../Data/from_sqlite_Data/Realfinal.csv')

In [69]:
final.drop(['Unnamed: 0'], axis=1,inplace=True)

In [ ]:
final

# For Betting !

In [278]:
a=pd.read_csv('../Data/premier_legue_old_data/E0_2017.csv')
season=a


In [279]:
#take the columns you care

In [280]:
season=season.iloc[:,1:7]

In [281]:
len(season.HomeTeam.unique())

20

In [282]:
#add the round to be predicted with 0 in all other values

In [283]:
round_to_be_predict=pd.DataFrame(columns=season.columns)

In [284]:
date=pd.to_datetime('28/04/17')

In [285]:
round_to_be_predict.loc[0]=[date,'Southampton','Hull',0,0,'D']
round_to_be_predict.loc[1]=[date,'Stoke','West Ham',0,0,'D']
round_to_be_predict.loc[2]=[date,'Sunderland','Bournemouth',0,0,'D']
round_to_be_predict.loc[3]=[date,'West Brom','Leicester',0,0,'D']
round_to_be_predict.loc[4]=[date,'Crystal Palace','Burnley',0,0,'D']
round_to_be_predict.loc[5]=[date,'Man United','Swansea',0,0,'D']
round_to_be_predict.loc[6]=[date,'Everton','Chelsea',0,0,'D']
round_to_be_predict.loc[7]=[date,'Middlesbrough','Man City',0,0,'D']
round_to_be_predict.loc[8]=[date,'Tottenham','Arsenal',0,0,'D']
round_to_be_predict.loc[9]=[date,'Watford','Liverpool',0,0,'D']



In [286]:
season.tail(n=10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
324,22/04/17,Hull,Watford,2,0,H
325,22/04/17,Swansea,Stoke,2,0,H
326,22/04/17,West Ham,Everton,0,0,D
327,23/04/17,Burnley,Man United,0,2,A
328,23/04/17,Liverpool,Crystal Palace,1,2,A
329,25/04/17,Chelsea,Southampton,4,2,H
330,26/04/17,Arsenal,Leicester,1,0,H
331,26/04/17,Crystal Palace,Tottenham,0,1,A
332,26/04/17,Middlesbrough,Sunderland,1,0,H
333,27/04/17,Man City,Man United,0,0,D


In [287]:
round_to_be_predict.Date=pd.to_datetime(round_to_be_predict.Date)

In [288]:
season=pd.concat([season,round_to_be_predict]).reset_index()

In [289]:
season.drop('index',axis=1,inplace=True)

In [290]:
season['season']=['2016/2017']*len(season)


In [291]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

In [293]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [ ]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_TP3',
                          'H_TP5',
                            
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',
    
                          'A_TP3',
                          'A_TP5',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [294]:
allteamsresults[0].df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,Round,T_L3M,T_L5M,GS_H,GS_A,GR_H,GR_A,TGS,TGR,AGS_H,AGS_A,AGR_H,AGR_A,ATGS,ATGR,Pnt_H,Pnt_A,TOTPT,Av_Pnt_H,Av_pnt_A,Av_tot_pnt
0,13/08/16,Burnley,Swansea,0.0,1.0,A,2016/2017,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000
11,20/08/16,Burnley,Liverpool,2.0,0.0,H,2016/2017,2,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0,0,0,0.000000,0.000000,0.000000
20,27/08/16,Chelsea,Burnley,3.0,0.0,H,2016/2017,3,3,3,2.0,0.0,1.0,0.0,2.0,1.0,1.000000,0.000000,0.500000,0.000000,1.000000,0.500000,3,0,3,1.500000,0.000000,1.500000
32,10/09/16,Burnley,Hull,1.0,1.0,D,2016/2017,4,0,0,2.0,0.0,1.0,3.0,2.0,4.0,1.000000,0.000000,0.500000,3.000000,0.666667,1.333333,3,0,3,1.500000,0.000000,1.000000
43,17/09/16,Leicester,Burnley,3.0,0.0,H,2016/2017,5,4,1,3.0,0.0,2.0,3.0,3.0,5.0,1.000000,0.000000,0.666667,3.000000,0.750000,1.250000,4,0,4,1.333333,0.000000,1.000000
59,26/09/16,Burnley,Watford,2.0,0.0,H,2016/2017,6,1,0,3.0,0.0,2.0,6.0,3.0,8.0,1.000000,0.000000,0.666667,3.000000,0.600000,1.600000,4,0,4,1.333333,0.000000,0.800000
66,02/10/16,Burnley,Arsenal,0.0,1.0,A,2016/2017,7,4,7,5.0,0.0,2.0,6.0,5.0,8.0,1.250000,0.000000,0.500000,3.000000,0.833333,1.333333,7,0,7,1.750000,0.000000,1.166667
78,16/10/16,Southampton,Burnley,3.0,1.0,H,2016/2017,8,3,4,5.0,0.0,3.0,6.0,5.0,9.0,1.000000,0.000000,0.600000,3.000000,0.714286,1.285714,7,0,7,1.400000,0.000000,1.000000
82,22/10/16,Burnley,Everton,2.0,1.0,H,2016/2017,9,3,4,5.0,1.0,3.0,9.0,6.0,12.0,1.000000,0.333333,0.600000,3.000000,0.750000,1.500000,7,0,7,1.400000,0.000000,0.875000
91,29/10/16,Man United,Burnley,0.0,0.0,D,2016/2017,10,3,6,7.0,1.0,4.0,9.0,8.0,13.0,1.166667,0.333333,0.666667,3.000000,0.888889,1.444444,10,0,10,1.666667,0.000000,1.111111


In [296]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[7:]) + list(allteamsresults[away_index].df.loc[i].iloc[7:])
    tmp.loc[i]=tobeappend

In [321]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)


final['DIFF_TP']=final['H_T_pnt']-final['Aw_T_pnt']
final['DIFF_HP-AP']=final['H_Pnt_H']-final['Aw_Pnt_A']

final['SUMM_HGSH+HGSH+AGRA']=final['H_GS_H']+final['Aw_GR_A']
final['SUMM_HGRH+AGSA']=final['H_GR_H']+final['Aw_GS_A']
final['SUMM_HTGS+ATGR']=final['H_TGS']+final['Aw_TGR']
final['SUMM_HTGR+ATGS']=final['H_TGR']+final['Aw_TGS']

final['DIFF_L3MP']=final['H_TP3']-final['A_TP3']
final['DIFF_L5MP']=final['H_TP5']-final['A_TP5']


final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']

final['Goal/No_Goal'] = ((final['FTHG']>0 ) * 1)*((final['FTAG']>0) * 1)
final['Over2.5/Under2.5'] = ((final['FTHG'] + final['FTAG']) >2)*1
final['Over1.5/Under1.5'] = ((final['FTHG'] + final['FTAG']) >1)*1



In [331]:
final.to_csv('../Data/from_sqlite_Data/***.csv')